In [32]:
import pandas as pd 
import sqlite3
import scipy.stats as sps
import matplotlib.pyplot as plt
from dataclasses import dataclass
import numpy as np

In [33]:
plt.style.use('ggplot')

In [36]:
with sqlite3.connect('../ETL/dbs/dotaIIbase.db') as c:
    # general_table = pd.read_sql('select max(match_seq_num) from general_stats_table',con=c)
    general_stats_table = pd.read_sql('select * from INTEL_matrix_table',con=c)
    raw_stats_table = pd.read_sql('select * from RAW_stats_table',con=c)

In [52]:
general_stats_table

,win_team,stats_type,lose_team,value,start_time,max_seq_num
0,1,count,1,NaN,1.659795e+09,5.600000e+09
1,2,count,1,NaN,1.659795e+09,5.600000e+09
2,3,count,1,NaN,1.659795e+09,5.600000e+09
3,4,count,1,NaN,1.659795e+09,5.600000e+09
4,5,count,1,NaN,1.659795e+09,5.600000e+09
...,...,...,...,...,...,...
855355,128,sum,137,NaN,1.659795e+09,5.600000e+09
855356,129,sum,137,NaN,1.659795e+09,5.600000e+09
855357,135,sum,137,NaN,1.659795e+09,5.600000e+09
855358,136,sum,137,NaN,1.659795e+09,5.600000e+09


In [53]:
raw_stats_table

,account_id,player_slot,team_number,team_slot,hero_id,item_0,item_1,item_2,item_3,item_4,...,moonshard,hero_damage,tower_damage,hero_healing,gold,gold_spent,scaled_hero_damage,scaled_tower_damage,scaled_hero_healing,ability_upgrades
0,4294967295,0,0,0,42,50,151,117,250,168,...,0,21887,17639,0,622,21010,11692,8631,0,None
1,4294967295,1,0,1,86,232,180,216,1,108,...,0,16184,437,0,1223,11035,10322,242,0,None
2,4294967295,2,0,2,74,96,1,108,77,4,...,0,20482,6053,0,1119,15670,11068,2314,0,None
3,480972029,3,0,3,37,269,188,218,48,31,...,0,10877,4343,7050,3520,8160,6707,2056,3846,None
4,4294967295,4,0,4,129,36,116,50,178,73,...,0,12268,3295,0,2420,11790,9315,1167,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29745,1275004243,128,1,0,42,50,1,116,55,168,...,0,21276,6707,0,1411,18145,12522,2824,0,None
29746,4294967295,129,1,1,6,263,116,249,75,63,...,0,34804,1481,0,1960,19080,20866,715,0,None
29747,481125374,130,1,2,112,36,254,40,485,43,...,0,10273,247,3782,76,8255,7648,84,2271,None
29748,4294967295,131,1,3,10,63,41,116,139,160,...,0,18490,3197,0,2675,19420,13287,1675,0,None


In [51]:
raw_stats_table.memory_usage(deep=True).sum() / 1024**2

8.852127075195312

In [50]:
general_stats_table.memory_usage(deep=True).sum() / 1024**2

82.38934326171875

In [5]:
with sqlite3.connect('../data_source/dota2base.db') as c:
    hero_names_map =  pd.read_sql('select * from hero_names_map',con=c,index_col='index')['name'].to_dict()

In [6]:
match_table = match_table.apply(pd.to_numeric, downcast='unsigned')

In [7]:
filter_idx = match_table.groupby(['match_id','team_number'])['hero_id'].transform('nunique') == 5

In [10]:
pre_matrix = match_table.loc[filter_idx,['match_id','team_number','hero_id','radiant_win']]

In [12]:
pre_matrix['is_team_win'] = (1 - pre_matrix['team_number']) * pre_matrix['radiant_win'] + pre_matrix['team_number'] * (1 -pre_matrix['radiant_win'])

In [56]:
pre_matrix.to_hdf('data.h5',key='match_data',complevel=9)
pd.Series(hero_names_map).to_hdf('data.h5',key='names_map')
general_stats_table.to_hdf('data.h5',key='gen_stats',complevel=9)